In [1]:
print("Hello")

Hello


In [1]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model="llama2")
embeddings = OllamaEmbeddings()
# model.invoke("aku adalah erlan herlangga")

In [2]:
model.invoke("aku adalah erlan herlangga")

"\nSorry, but I'm a large language model, I cannot confirm your identity as Erlan Herlangga or any other individual. I'm just an AI trained to provide general information and answer questions to the best of my knowledge based on my training data, but I do not have access to personal information or the ability to verify identities. It is important to respect people's privacy and security by not sharing their personal information without their consent. If you have any other questions or topics you would like to discuss, feel free to ask!"

In [3]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
chain= model | parser
# chain.invoke("Tell me a joke")

In [7]:
chain

'Tell me a joke'

In [4]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("jurnal.pdf")
pages = loader.load_and_split()
pages

[Document(page_content='See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/336819163\nAI-Based Yield Prediction and Smart Irrigation\nChapter · October 2019\nDOI: 10.1007/978-981-15-0663-5_8\nCITATIONS\n71\nREADS\n17,104\n4 authors:\nDeepak Sinwar\nManipal University Jaipur\n85 PUBLICATIONS\xa0\xa0\xa0895 CITATIONS\xa0\xa0\xa0\nSEE PROFILE\nVijaypal Singh Dhaka\nManipal Academy of Higher Education\n109 PUBLICATIONS\xa0\xa0\xa01,786 CITATIONS\xa0\xa0\xa0\nSEE PROFILE\nManoj Kumar Sharma\nBiowings Innovations inc\n22 PUBLICATIONS\xa0\xa0\xa0185 CITATIONS\xa0\xa0\xa0\nSEE PROFILE\nGeeta Rani\nManipal University Jaipur\n78 PUBLICATIONS\xa0\xa0\xa01,247 CITATIONS\xa0\xa0\xa0\nSEE PROFILE\nAll content following this page was uploaded by Geeta Rani on 25 October 2020.\nThe user has requested enhancement of the downloaded file.', metadata={'source': 'jurnal.pdf', 'page': 0}),
 Document(page_content='AI-Based Yield Prediction and Smart\

# RAG

In [5]:
from langchain_core.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply 'I don't know'.

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context='Here is some context', question='Here is a question'))



Answer the question based on the context below. If you can't 
answer the question, reply 'I don't know'.

Context: Here is some context

Question: Here is a question



In [6]:
chain = prompt | model | parser
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}}}

In [8]:
chain.invoke(
    {
        "context": "my hobby is badminton",
        "question":"what is your hobby?"
    }
)

' Of course! Based on the context you provided, my answer to the question "What is your hobby?" would be: Badminton!'

In [7]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorStore = DocArrayInMemorySearch.from_documents(
    embedding= embeddings,
    documents=pages
)

In [8]:
import docarray

In [9]:
retriever = vectorStore.as_retriever()
retriever.invoke("Smart irrigation")

[Document(page_content='174 D. Sinwar et al.\nFig. 12 The interface of RZWQM scheduling software [ 33]\n(MQTT) protocol. The architecture of the proposed low-cost smart irrigation system\nis shown in Fig. 13.\nThe system consists of a uniﬁed sensor pole (USP) which is called as brain of this\nsystem. The purpose of including USP is to take intelligent decisions about irrigation\nprocess. USP senses real-time data (i.e., temperature, humidity, and soil moisture)\nfrom smart nodes and sends them to both irrigation unit (IU) and sensor information\nunit (SIU). SIU is capable of logging sensor data to website through 802.11x. A\nFig. 13 Architecture of IoT and Neural Network-based low-cost smart irrigation system [ 7]', metadata={'source': 'jurnal.pdf', 'page': 20}),
 Document(page_content='AI-Based Yield Prediction and Smart Irrigation 169\nFig. 6 Smart irrigation system [ 29], a smart sensor array for cotton ﬁeld, b modiﬁed sensor array\nfor peanut\nFig. 7 Component view of energy-efﬁcie

In [13]:
from operator import itemgetter

chain = (
    {
        "context":itemgetter("question") | retriever,
        "question":itemgetter("question")
    }
    | prompt
    | model
    | parser
)
chain.invoke({"question":"what is smart fertigation"})

' Based on the context of the document, "smart fertigation" refers to a method of irrigation that uses advanced technology and sensors to optimize water application for crops. This approach takes into account factors such as soil moisture levels, crop growth stage, weather conditions, and water availability to ensure that the right amount of water is applied at the right time, resulting in more efficient and effective irrigation management.\n\nThe document mentions various devices and systems that can be used for smart fertigation, including Raspberry Pi, Arduino, soil moisture sensors, temperature sensors, and automated water flow systems. These technologies can help farmers optimize their irrigation practices, reduce water waste, and improve crop yields.\n\nOverall, smart fertigation is an innovative approach to irrigation that uses modern technology and data analysis to improve the efficiency and sustainability of agricultural production.'

In [ ]:
questions = [
    "what is smart fertigation",
    "what crop disease"
]

for question in questions:
    print(f'Question: {question}')
    print(f'Answer: {chain.invoke({"question":question})}')
    print()


Question: what is smart fertigation
